<a href="https://colab.research.google.com/github/HanRunbing/Tool_boxes/blob/main/image_analysis_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import io

In [ ]:
dapi = io.imread("/content/drive/MyDrive/coding_practise/image_analysis/dapi.tif")
membrane= io.imread("/content/drive/MyDrive/coding_practise/image_analysis/membrane.tif")
print(dapi.shape)
print(membrane.shape)

<tifffile.TiffFile 'dapi.tif'> OME series is BinaryOnly, not an OME-TIFF master file 
<tifffile.TiffFile 'membrane.tif'> OME series is BinaryOnly, not an OME-TIFF master file 


(5, 2960, 2960)
(5, 2960, 2960, 4)


# Normalization & MIP

For np.array.max is to find the max indensity for each pixel across all Zs.

In [ ]:
demo0 = np.array([[0,1,2,3],[10,2,8,4],[2,3,4,5]])
demo0_max =  demo0.max(axis = 0)
demo0_max

array([10,  3,  8,  5])

For np.quantile is to return value of 0.01 position and ... even the value is not in the array 

In [ ]:
np.quantile(demo0_max, [0.01, 0.99])


array([3.06, 9.94])

min max mormalization 

In [ ]:
lower, upper = np.quantile(demo0_max, [0.01, 0.99])
demo0_normal = (demo0_max - lower)/upper
demo0_normal

array([ 1.00872093, -0.00872093,  0.71802326,  0.28197674])

Thus for normalize and MIP dapi:

In [ ]:
# get the max value across all Zs, the output may not come from the same z
d = dapi.max(axis = 0)  
# return the low 1% and high intensity 99%
lower, upper = np.quantile(d, [0.01, 0.99])
# normalize:
d = (d - lower)/upper
d_normalized = d 
d_normalized.shape

(2960, 2960)

In [ ]:
m_normalized = []
for i in range(3): 
  # process normalizaiton for each channels
    m = membrane[:,:,:,i].max(axis = 0) # i+ 1 for first channel is emply
    lower, upper = np.quantile(m, [0.01, 0.99])
    m = (m - lower)/upper
    m_normalized.append(m)
# normalization are done for each channel separatly, stack baxk to 3 channels
m_normalized = np.stack(m_normalized) 
m_normalized.shape

(3, 2960, 2960)

# Path operation

Grap all files in the folder with certain type and replace its name ot switch to another type

In [ ]:
xs = []
pos_t = []
# with glob to get the filenames, * to match everthing
for dapi_path in glob.glob("../Segmentation_test/1208_anti_spot_9/DAPI/*.tif"):
  #   replace the name with replace
    membrane_path = dapi_path.replace('DAPI', 'membrane').replace('STAIN', 'hyb10')
  # get the specific name of file 
    pos = os.path.basename(dapi_path).split('.ome')[0].split('Pos')[-1]

    # load data
    dapi = io.imread(dapi_path)
    dapi = np.stack([np.stack(cv2.resize(dapi[i,:,:], (512,512))) for i in range(5)])
    membrane= io.imread(membrane_path)
    membrane = np.stack([np.stack(cv2.resize(membrane[i,:,:], (512,512))) for i in range(20)])
    membrane = membrane.reshape(20, 512, 512).reshape(4, 5, 512, 512) 
    
    merged = mergeGen(membrane,dapi)
    
    xs.append(merged)
    pos_t.append(pos)
    
xs = np.stack(xs)

In [ ]:
list0 = range(3,8)
for i, x in enumerate(list0):
  print(i,x)

0 3
1 4
2 5
3 6
4 7


# Plot template

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))

img = viz_mip_membrane_dapi.copy()
ax[0].imshow(img)
ax[0].set_title('r = hyb, g = mem, b = dapi')

img = viz_mip_hyb_dapi.copy()
ax[1].imshow(img)
ax[1].set_title('r = epi, b = fib, g = immune')

img = gray2rgb(rgb2gray(viz_mip_membrane_dapi))
ax[2].imshow(img)
ax[2].scatter(seg_4fov.loc[select, 'x'], seg_4fov.loc[select, 'y'], s = 0.5, 
    c = seg_4fov.loc[select,'cluster_color'])
ax[2].set_title('scatter = baysor ncv cluster')

ax[0].set_xlim(0, 5500)
ax[0].set_ylim(5500, 0)
ax[1].set_xlim(0, 5500)
ax[1].set_ylim(5500, 0)
ax[2].set_xlim(0, 5500)
ax[2].set_ylim(5500, 0)

plt.tight_layout()